# パラメータ実験用のディレクトリを追加する

このタスクでは、データセットの構造として、パラメータ実験用のデータ構成(RCOS_for_parameters)を指定した研究者向けに、  
実験パッケージ内に、パラメータ実験用のディレクトリを追加します。

<img src="./../../../files/WORKFROWS/EX-WORKFROWS/images/パラメータ実験用パッケージイメージ.png">

##  1. タスクの実行対象確認

パラメータ実験用のデータ構成を利用していない方にとっては不要なタスクとなります。  
下記のセルを実行し、本タスクの実行対象かご確認ください。

In [ ]:
import os
import json

path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
os.chdir(path_flow_root)
from util.scripts import utils

# DS構成のスキーム名をパラメタファイルから取得する
monitoring_params = {}
with open(utils.fetch_monitoring_param_file_path(), mode='r') as f:
    monitoring_params = json.load(f)
    
scheme_name = monitoring_params['datasetStructure']

if scheme_name != 'RCOS_for_parameters':
    print( '本処理は不要ですので、実験ワークフロー機能トップページに遷移してください。')

上記の処理で、「本処理は不要ですので、実験ワークフロー機能トップページに遷移してください。」と表示された方は、  
[こちら](../experiment.ipynb)からトップページに遷移できます。

## 2. パラメータ実験用のディレクトリを追加する

### 2-1. パラメータ実験用のディレクトリを用意する

In [ ]:
import os

# このコンテナで扱う実験パッケージのパスを作成する
%store -r EXPERIMENT_TITLE
experiment_path = '/home/jovyan/experiments/' + EXPERIMENT_TITLE

# /home/jovyan 配下に移動する
os.chdir(os.environ['HOME'])

# ディレクトリを追加する
!cp -r ~/WORKFLOWS/PACKAGE/scheme/$scheme_name/parameter $experiment_path


### 2-2. パラメータ実験名を決定する

パラメータ実験名を入力して、実験データ格納用のディレクトリを作成します。

パラメータ実験名としては、実験パッケージ内で同名のパラメータ実験名を使用することはできません。  
また、「parameter」というパラメータ実験名も使用することができません。パラメータ名など分かりやすい名前をご記入ください。  
※入力値に誤りがある場合、もう一度実行することで訂正ができます。  


In [ ]:
import os
import re
import shutil
from IPython.display import clear_output

# データセットの構成がパラメータ実験用ではない場合、何もしない。
if scheme_name != 'RCOS_for_parameters':
    print( '本処理は不要です。')
else:
    # GINサーバのものに合わせたバリデーションルールを設定
    validation = re.compile(r'[a-z|A-Z|0-9|\-|_|.]+')

    print('作成したいパラメータ実験名を半角英数字で入力してください。')
    while True:
        paramexp_title = input("パラメータ実験名：")
        if validation.fullmatch(paramexp_title):
            break
        else:
            clear_output()
            print('パラメータ実験名は英数字、および"-", "_", "."のみで入力してください。')

    clear_output()

    # 該当実験パッケージを特定させるため、環境変数EXPERIMENT_TITLEに実験パッケージ名を設定
    PARAMEXP_TITLE = paramexp_title
    %store PARAMEXP_TITLE
    clear_output()

    print("作成したパラメータ実験名：", paramexp_title)
    print('このパラメータ実験名で処理を進めます。')

    # 実験パッケージの直下に移動
    os.chdir(experiment_path)

    # parameterディレクトリをユーザが指定したパラメータ実験名に変更
    shutil.move('parameter', paramexp_title)

## 3. 本タスクの実行結果をGIN-forkに同期する

ここまでの内容を保存し、GIN-forkに同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import os

# 実験パッケージの直下へと移動する    
os.chdir(experiment_path)

#**************************************************#
#* Git-annex管理するディレクトリへのパスを生成する #
#**************************************************#
dirlist=[]
filelist=[]
annexed_save_path=[]

# 実験パッケージ配下を再帰的に検索し、ディレクトリとファイルの一覧を取得する。
for root, dirs, files in os.walk(top=experiment_path):
    # ディレクトリ情報から絶対パスを作成する
    for dir in dirs:
        dirPath = os.path.join(root, dir)
        dirlist.append( dirPath )   

# output_dataという文字列を含むディレクトリを検索する。
output_data_path = [ s for s in dirlist if 'output_data' in s ]

for output_data in output_data_path:  
    # input_data配下に含まれる、output_dataディレクトリは対象外
    if  "input_data" not in output_data:
        # 該当の文字列を含んだパスをGit-annex管理するパスの配列に追記する
        annexed_save_path.append( output_data )

# input_dataディレクトリをGit-Annex管理するパスの配列に追記する。
# input_dataは実験パッケージの直下にあることを想定。
# input_dataには、他リポジトリの情報が含まれる可能性があり、そのリポジトリの内容はGINに書き戻したくないため。
annexed_save_path.append( experiment_path + '/input_data'  )

#********************************************************#
#* Gitで管理するディレクトリとファイルへのパスを生成する #
#********************************************************#
#  実験パッケージ直下のディレクトリとファイルの一覧を取得する
files = os.listdir()

# 取得した一覧からGit-annex管理するディレクトリ(input_dataとoutput_data)を削除する
dirs = [f for f in files if os.path.isdir(f)]

for dirname in dirs:
    if dirname == 'input_data' :
        dirs.remove('input_data')

    if dirname == 'output_data' :
        dirs.remove('output_data')
    
# 実験パッケージ直下のファイルを取得
files = [f for f in files if os.path.isfile(f)]

# Git管理するパスの配列を作成する
files.extend(dirs)
save_path = []
for file in files:
    save_path.append(experiment_path + '/' + file)
save_path.append('/home/jovyan/pipeline.json')

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](http://dg02.dg.rcos.nii.ac.jp/G-Node/Info/wiki/%E3%83%AF%E3%83%BC%E3%82%AF%E3%83%95%E3%83%AD%E3%83%BC#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import os
import papermill as pm
from colorama import Fore
from IPython.display import clear_output

os.chdir(os.environ['HOME'])
# Git-annex管理ファイルを保存
try:
    pm.execute_notebook(
        'WORKFLOWS/EX-WORKFLOWS/util/base_datalad_save_push.ipynb',
        '/home/jovyan/.local/push_log.ipynb',
        parameters = dict(SAVE_MESSAGE = EXPERIMENT_TITLE + '_パラメータ実験準備 (1/2)', PATH = annexed_save_path, IS_RECURSIVE = False, UNLOCK = False, REMOVE_PUSH_PATH = True)
    )
finally:
    clear_output()
    %store -r DATALAD_MESSAGE
    %store -r DATALAD_ERROR
    print('\n' + DATALAD_MESSAGE + '\n')
    print(Fore.RED + DATALAD_ERROR)

In [ ]:
import os
import papermill as pm
from colorama import Fore
from IPython.display import clear_output

os.chdir(os.environ['HOME'])
# Git管理ファイルを保存
try:
    pm.execute_notebook(
        'WORKFLOWS/EX-WORKFLOWS/util/base_datalad_save_push.ipynb',
        '/home/jovyan/.local/push_log.ipynb',
        parameters = dict(SAVE_MESSAGE = EXPERIMENT_TITLE + '_パラメータ実験準備 (2/2)', TO_GIT = True, PATH = save_path, IS_RECURSIVE = False)
    )
finally:
    clear_output()
    %store -r DATALAD_MESSAGE
    %store -r DATALAD_ERROR
    print('\n' + DATALAD_MESSAGE + '\n')
    print(Fore.RED + DATALAD_ERROR)

## 4. 実験ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../experiment.ipynb)からトップページに遷移できます。  